In [1]:
print "OK"

"hello"
%load_ext autoreload
#%autoreload 2
import pandas 
import pandas as pd 
import sklearn
from backend import db_connection2 as db
from backend import db_connection2 
import sys

OK


In [2]:
conn, meta, session = db.connect("postgres", "1234", db="match_stick_girl") #temp2

In [3]:
all_text_manipulations=session.query(db.Process_Text_Manipulation).all()

#baby_born
#babay
#rewrite_task2

#For all Process Rewrite
#For all COntent whose origin id == Process_Rewrite
#Where User_ID is > -1 #not null

#For process - hash_id process and estimated length 
#For content compeltion

In [4]:
one_el=all_text_manipulations[0]
#one_el=db_connection2.Process_Text_Manipulation
#one_el.task_parameters_obj

def time_estimate(process):
    total_text=""
    total_text+=" "+process.task_parameters_obj.body_of_task.results
    total_text+=" "+process.task_parameters_obj.prompt.results
    total_text+=" "+process.task_parameters_obj.result.results
    total_text+=" "+process.task_parameters_obj.suggestion.results
    total_text+=" "+process.task_parameters_obj.context.results
    words=len(total_text.split(" "))
    time = words/ 5.5

    return time  

print time_estimate(all_text_manipulations[3])

20.7272727273


In [5]:
#estimate rating score
#estimate writing score 
rating_score_matrix=[]
writing_score_matrix=[] #writing score matrix first
databases=["match_stick_girl","baby_born","babay"]

In [6]:
for database in databases:
    conn, meta, session = db.connect("postgres", "1234", db=database) 
    
    process=session.query(db.Process_Text_Manipulation).subquery()
    content=session.query(db.Content).filter((process.c.id == 
                                             db.Content.origin_process_id)
                                             & (db.Content.is_completed==True)
                                             )
    content_subquery=content.subquery()
    
    
    task_params=session.query(db.Task_Parameters).filter\
        (content_subquery.c.id==db.Task_Parameters.result_id).subquery()
    
    process_ratings=session.query(db.Process_Rate).\
        filter(task_params.c.id==db.Process_Rate.task_parameters_id).all()
    
    #Time it took to read the task, time it took to complete, and overall score
    for i in process_ratings:
        if  len(i.get_final_results_complete(session).all()) > 0:
            result=i.task_parameters_obj.result 
    
            
            score= i.get_final_results_complete(session).all()[0].results 
            
            associated_process=session.query(db.Process_Text_Manipulation).filter\
                (result.origin_process_id==db.Process_Text_Manipulation.id).all()[0]
            time_to_read= time_estimate(associated_process)
            time_to_complete= (result.completed_date-result.assigned_date).total_seconds()
            writing_score_matrix.append(
                [float(score),time_to_read,time_to_complete])
            for j in i.get_content_produced_by_this_process():
                if j.is_completed == True:
                    time_to_read= time_estimate(i) 
                    avg_score=float(score)
                    score = -2
                    try:
                        score=float(j.results)
                    except: pass 
                    general_time_to_complete=time_to_complete 
                    time_to_complete=(j.completed_date-j.assigned_date).total_seconds()
                    time_to_read_result=len(result.results.split(" "))/5.5
                    rating_task=[score,avg_score,time_to_read,time_to_read_result,
                                 time_to_complete]
                    rating_score_matrix.append(rating_task)
        #print "end of content"

In [92]:
import pandas as pd 
import numpy as np 
ary=np.array(writing_score_matrix)
frame=pd.DataFrame(data=ary,columns=["Score","Time_To_Read","Time_To_Complete"])
frame["ratio"]=frame.Time_To_Complete/frame.Time_To_Read
function=lambda t: "Bad" if t < 3 else "Good"
frame["binary_score"]=frame['Score'].apply(function)

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

names=preprocessing.LabelEncoder()
names.fit(frame['binary_score'])

LabelEncoder()

In [146]:
from sklearn.metrics import r2_score
from sklearn import tree 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#r=tree.DecisionTreeClassifier(criterion="entropy",min_samples_split=10)
#r=RandomForestClassifier(min_samples_split=10)
r=LogisticRegression()
#r=KNeighborsClassifier(n_neighbors=5)
p=make_pipeline( r) #PolynomialFeatures(1)

#p=LogisticRegression()

X=frame[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame["binary_score"]
X, y = balanced_subsample(X.values,names.transform(y))
p.fit(X,y)

frame_pos=frame #frame[frame.Score>0] #[frame.binary_score=="Good"]
X=frame_pos[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame_pos["binary_score"]

predicted_y= p.predict(X.values)
#print y 
#print predicted_y
print p.score(X.values,names.transform(y.values) ) 
print len(y.values)



#print p.coef_
#print p.get_params()
predicted_prob= p.predict_proba(X.values)
#print r.feature_importances_
ground_truth_y_predicted_prob=[]
print "classes"
print r.classes_
for i,j in enumerate(y):
    
    location= names.transform([j])[0]
    #print "name"
    #print j 
    #print location
    #print r.classes_
    #print predicted_prob[i]
    #print predicted_prob[i][location]
    ground_truth_y_predicted_prob.append(predicted_prob[i][location])
#good set
predicted=p.predict(X)
#print np.array(ground_truth_y_predicted_prob)
prc= precision_recall_curve(names.transform(y.values),
                       ground_truth_y_predicted_prob)
print len(prc[0])
print len(prc[1])
print len(prc[2])
print pandas.DataFrame(data={"pr":prc[0][:-1],"rc":prc[1][:-1],"thr":prc[2]})
confusion_matrix(names.transform(y.values),predicted)

0.64367816092
87
classes
[ 0.  1.]
87
87
86
          pr        rc       thr
0   0.627907  1.000000  0.244452
1   0.623529  0.981481  0.253576
2   0.630952  0.981481  0.293662
3   0.626506  0.962963  0.312099
4   0.621951  0.944444  0.314092
5   0.617284  0.925926  0.315391
6   0.625000  0.925926  0.321579
7   0.620253  0.907407  0.327707
8   0.628205  0.907407  0.332898
9   0.623377  0.888889  0.334613
10  0.618421  0.870370  0.338780
11  0.626667  0.870370  0.350090
12  0.621622  0.851852  0.366713
13  0.616438  0.833333  0.368317
14  0.611111  0.814815  0.380042
15  0.605634  0.796296  0.382676
16  0.600000  0.777778  0.383393
17  0.594203  0.759259  0.386440
18  0.588235  0.740741  0.386908
19  0.582090  0.722222  0.393832
20  0.575758  0.703704  0.398644
21  0.584615  0.703704  0.408980
22  0.578125  0.685185  0.413098
23  0.587302  0.685185  0.416830
24  0.580645  0.666667  0.425641
25  0.573770  0.648148  0.430411
26  0.566667  0.629630  0.450181
27  0.559322  0.611111  0.455239

array([[25,  8],
       [23, 31]])

In [237]:
#"Time_To_Read","Time_To_Complete","ratio"
time_to_read=60
time_to_complete=35
ratio=time_to_complete/time_to_read
p.predict_proba([[time_to_read,time_to_complete,ratio]])

#if score > 3
# 1- no_prob * score versus
#

array([[ 0.82845816,  0.17154184]])

In [60]:
from sklearn.model_selection import KFold
k=KFold(n_splits=2)
#r=tree.DecisionTreeClassifier(min_samples_split=10)
#p=make_pipeline(PolynomialFeatures(1), r)
frame_subset=frame #[frame["binary_score"]=="Bad"]
X=frame_subset[["Time_To_Read","Time_To_Complete","ratio"]]
y=frame_subset[["binary_score"]]
for train,test in k.split(X.values,y.values):
    train_X=X.values[train]
    train_y=y.values[train]
    train_X,train_y=balanced_subsample(train_X,names.transform(train_y))
    test_y=names.transform(y.values[test])
    test_X=X.values[test]
    p.fit(train_X,train_y)
    print p.score(test_X,test_y)
    print confusion_matrix(test_y,p.predict(test_X))

0.545454545455
[[13  5]
 [15 11]]
0.581395348837
[[10  5]
 [13 15]]


In [239]:
import pandas as pd 
import numpy as np 
ary=np.array(rating_score_matrix)

frame=pd.DataFrame(data=ary,columns=["Score","Avg_Score","Time_To_Read",
                                     "Time_To_Read_Result",
                                     "Time_To_Complete"])

frame["ratio"]=frame.Time_To_Complete/frame.Time_To_Read
frame["score_diff"]=(frame.Avg_Score-frame.Score).abs()
function=lambda t: "Bad" if t > 2 else "Good"
frame["binary_score"]=frame['score_diff'].apply(function)


names=preprocessing.LabelEncoder()
names.fit(frame['binary_score'])

LabelEncoder()

In [240]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn import tree

r=tree.DecisionTreeClassifier(min_samples_split=10)
p=make_pipeline( r) #PolynomialFeatures(1),

#p=LogisticRegression()
frame=frame #[frame["score_diff"]>0]
X=frame[["Time_To_Read","Time_To_Read_Result","Time_To_Complete","ratio"]]
y=frame["binary_score"]

p.fit(X.values,y.values)

frame_pos=frame#[frame.binary_score=="Bad"]
X=frame_pos[["Time_To_Read","Time_To_Read_Result","Time_To_Complete","ratio"]]
y=frame_pos["binary_score"]
predicted_y= p.predict(X.values)
#print y 
#print predicted_y
#print p.score(X.values,y.values) 
#print mean_absolute_error(y,predicted_y)
#print mean_squared_error(y,predicted_y)
#print len(predicted_y)


#print pandas.DataFrame(data={"y_hat": predicted_y, "y":y})

In [241]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
splits=2
k=KFold(n_splits=splits,shuffle=True)
r=RandomForestRegressor(min_samples_split=5)
#r=Ridge()
#r=RandomForestRegressor(min_samples_split=5)
r=SVR()#KNeighborsRegressor(n_neighbors=10)

#r=tree.DecisionTreeClassifier(min_samples_split=10)
#p=make_pipeline(PolynomialFeatures(1), r)
frame_subset=frame #[frame["binary_score"]=="Bad"]
X=frame_subset[["Score","ratio","Time_To_Read","Time_To_Read_Result","Time_To_Complete"]]
y=frame_subset["score_diff"]

X_transformed_scaler=StandardScaler()
X_transformed_scaler.fit(X.values)
p=make_pipeline(X_transformed_scaler, r)
#print X
total_score=0
total_abs_error=0
total_squared_err=0

#pandas.DataFrame(data={"x":X["ratio"],"y":y}).to_csv("graph_data.csv")

for train,test in k.split(X.values,y.values):
    train_X=X.values[train]
    train_y=y.values[train]
  
    test_y=y.values[test]
   
    test_X=X.values[test]
    
    #print len(test_y)
    #print len(train_y)
    p.fit(train_X,train_y)
    #print r.decision_path(test_X)
    total_score+= p.score(test_X,test_y)
   
    predicted_y=p.predict(test_X)
    
    print p.score(test_X,test_y)
   
    total_abs_error+= mean_absolute_error(test_y,predicted_y)
    total_squared_err+= mean_squared_error(test_y,predicted_y)
#print r.feature_importances_
print total_score/float(splits),
print total_abs_error/float(splits),
print total_squared_err/float(splits)


0.0340948214441
0.14890525501
0.0915000382271 0.973833735579 1.92607200783
next


In [242]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys